In [1]:
from keras.models import * 
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.utils.training_utils import multi_gpu_model
from multiprocessing import cpu_count
import tensorflow as tf

Using TensorFlow backend.


In [2]:
nb_classes = 20
nb_cpus = cpu_count()//2
nb_gpus = 4

image_size = (299, 299)
input_shape= (299, 299, 3)

In [5]:
with tf.device('/cpu:0'):
    input_tensor = Input(input_shape)
    x = Lambda(xception.preprocess_input)(input_tensor)

    base_model = Xception(input_tensor=x, weights=None, include_top=False)
    m_out = base_model.output
    p_out = GlobalAveragePooling2D()(m_out)
    p_out = Dropout(1.0)(p_out)
    predictions = Dense(nb_classes, activation='softmax')(p_out)
    model = Model(inputs=base_model.input, outputs=predictions)
    
model = multi_gpu_model(model, gpus=nb_gpus)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_7 (Lambda)               (None, 299, 299, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
lambda_8 (Lambda)               (None, 299, 299, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
lambda_9 (Lambda)               (None, 299, 299, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
lambda_10 

In [6]:
model.load_weights('weights_train5_010.hdf5')

In [7]:
batch_size = 128
test_path = "/home/cnn/Documents/batch6.1/train5/valid"

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory(test_path, 
                                         target_size=image_size, 
                                         shuffle=False,
                                         batch_size=batch_size)
test_img_nums = test_generator.samples
all_test_results = model.predict_generator(test_generator, 
                                           len(test_generator), 
                                           workers=nb_cpus, 
                                           use_multiprocessing=True,
                                           verbose=1)
all_labels = test_generator.classes

Found 25362 images belonging to 20 classes.
199/199 [==============================] - 108s 543ms/step


In [8]:
class_label_dict = test_generator.class_indices
print(test_generator.class_indices)
def get_key(dict_, value):
    return [k for k, v in dict_.items() if v == value]

# create class num lens dict, every dict store current class predict num
total_predictions_dict = {}
for row_class_name, label in class_label_dict.items():
    total_predictions_dict[row_class_name] = {}
    for column_class_name, label in class_label_dict.items():
        total_predictions_dict[row_class_name][column_class_name] = 0

{'EC': 5, 'AGC_A': 1, 'ASCUS': 3, 'VIRUS': 19, 'LSIL_F': 12, 'SCC_R': 17, 'AGC_B': 2, 'SCC_G': 16, 'GEC': 7, 'RC': 14, 'MC': 13, 'HSIL_B': 8, 'CC': 4, 'ACTINO': 0, 'LSIL_E': 11, 'HSIL_M': 9, 'TRI': 18, 'FUNGI': 6, 'SC': 15, 'HSIL_S': 10}


In [19]:
thresh = 0.0

for i, label in enumerate(all_labels):
    predict_index = np.argmax(all_test_results[i])
    predict_det = all_test_results[i][predict_index]
    if (predict_det > thresh):
        # get the first result
        label_class_name = get_key(class_label_dict, label)[0]
        test_class_name = get_key(class_label_dict, np.argmax(all_test_results[i]))[0]
#     print(label_class_name)
#     print(total_predictions_dict[label_class_name].keys())
        total_predictions_dict[label_class_name][test_class_name] += 1

In [20]:
print(total_predictions_dict['RC'])

{'EC': 3, 'AGC_A': 0, 'ASCUS': 0, 'LSIL_F': 0, 'VIRUS': 0, 'SCC_R': 0, 'AGC_B': 0, 'SCC_G': 0, 'GEC': 7, 'HSIL_S': 0, 'RC': 282, 'ACTINO': 0, 'HSIL_B': 0, 'CC': 0, 'MC': 0, 'LSIL_E': 0, 'TRI': 0, 'HSIL_M': 0, 'SC': 0, 'FUNGI': 0}


In [21]:
import csv
out = open('confusion_matrix.csv','a', newline='')
csv_write = csv.writer(out,dialect='excel')

# write the title
line = [class_name for class_name, label in class_label_dict.items()]
line = [" "] + line + ["TOTAL"] + ["ACC"]
csv_write.writerow(line)

# write rows

true_num = 0
all_num = 0

for row_class_name, label in class_label_dict.items():
    one_class_total_predict = 0
    line = [row_class_name]
    for column_class_name, label in class_label_dict.items():
        one_class_total_predict += total_predictions_dict[row_class_name][column_class_name]
    
    for column_class_name, label in class_label_dict.items():
        one_class_cur_predict = total_predictions_dict[row_class_name][column_class_name]
        # acc
        #acc = round((one_class_cur_predict / one_class_total_predict), 4)
        #line.append(str(acc))
        # num
        line.append(one_class_cur_predict)
    print(one_class_total_predict)
    print(str(one_class_total_predict))
    line.append(str(one_class_total_predict))
    line.append(round((total_predictions_dict[row_class_name][row_class_name] / one_class_total_predict), 4))       
    print(line)
    csv_write.writerow(line)
    
    true_num += total_predictions_dict[row_class_name][row_class_name]
    all_num += one_class_total_predict
    
csv_write.writerow(["ALL_ACC"] + [round((true_num / all_num), 4)])
    
out.close()

465
465
['EC', 437, 0, 0, 0, 0, 0, 8, 0, 18, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, '465', 0.9398]
1969
1969
['AGC_A', 0, 1912, 0, 0, 1, 0, 5, 17, 20, 0, 0, 10, 0, 0, 0, 4, 0, 0, 0, 0, '1969', 0.9711]
3599
3599
['ASCUS', 0, 0, 3392, 1, 79, 0, 0, 1, 2, 0, 12, 0, 1, 0, 48, 4, 0, 0, 0, 59, '3599', 0.9425]
2146
2146
['VIRUS', 0, 0, 0, 2116, 0, 0, 0, 0, 5, 0, 0, 0, 1, 0, 0, 1, 0, 0, 5, 18, '2146', 0.986]
840
840
['LSIL_F', 0, 0, 130, 0, 663, 0, 0, 4, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 40, '840', 0.7893]
3598
3598
['SCC_R', 0, 0, 0, 0, 1, 3569, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 24, '3598', 0.9919]
804
804
['AGC_B', 36, 11, 0, 0, 0, 0, 707, 0, 23, 1, 0, 14, 0, 0, 0, 12, 0, 0, 0, 0, '804', 0.8794]
2066
2066
['SCC_G', 0, 3, 10, 0, 14, 11, 0, 1864, 17, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 143, '2066', 0.9022]
3952
3952
['GEC', 8, 17, 6, 3, 1, 0, 16, 4, 3836, 14, 0, 20, 0, 3, 0, 14, 0, 0, 0, 10, '3952', 0.9706]
292
292
['RC', 3, 0, 0, 0, 0, 0, 0, 0, 7, 282, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '292', 0.9658]
6200